In [ ]:
from xarm.wrapper import XArmAPI
from applied_planning.hardware import XArmLite6Adapter  

ROBOT_IP = "192.168.1.161"
arm = XArmAPI(ROBOT_IP)
robot = XArmLite6Adapter(ROBOT_IP)

ROBOT_IP: 192.168.1.161, VERSION: v1.11.4, PROTOCOL: V1, DETAIL: 6,9,LI1003,DL1000,v1.11.4, TYPE1300: [0, 0]
change protocol identifier to 3


ControllerError, code: 2


In [13]:
robot.reset() # move robot to home position

[set_state], xArm is ready to move
Moving to home position...
✓ Home position reached


{'qpos': array([ 2.17827004e-01,  5.39807997e-01,  5.19227005e-01, -3.99680399e-06,
        -2.05809924e-02,  2.17831996e-01]),
 'qvel': array([0., 0., 0., 0., 0., 0.])}

In [ ]:
import time
import math
import numpy as np

print("Moving robot in circular pattern with sinusoidal vertical motion...")
print("Press Ctrl+C to stop")

radius = 0.08  # 8cm radius (was 15cm - too big!)
center_x = 0.25  # 25cm in front (was 30cm+15cm = 45cm - too far!)
center_y = 0.0  # Centered
base_z = 0.15  # 15cm height (was 20cm - may be too low for some positions)
z_amplitude = 0.03  # 3cm vertical wave (was 5cm)

duration = 30
num_points = 60  # Fewer points = smoother motion
speed = 50  # Slower speed for safety (was 100)

try:
    # Get current position
    code, current_pos = robot.arm.get_position()
    print(f"Current position: {current_pos[:3]}")
    
    # Enable motion
    robot.arm.set_mode(0)
    robot.arm.set_state(0)
    time.sleep(0.5)
    
    # Move to starting position first
    start_x = (center_x + radius) * 1000
    start_y = center_y * 1000
    start_z = base_z * 1000
    
    print(f"Moving to start position: ({start_x:.1f}, {start_y:.1f}, {start_z:.1f})mm")
    code = robot.arm.set_position(
        x=start_x, y=start_y, z=start_z,
        roll=180, pitch=0, yaw=0,
        speed=100, wait=True
    )
    
    if code != 0:
        print(f"ERROR: Could not reach start position! Code={code}")
        print("Try adjusting center_x, center_y, or base_z")
        raise ValueError(f"Cannot reach start position, error code: {code}")
    
    print("Starting circular motion...")
    
    for i in range(num_points):
        # Calculate positions
        angle = 2 * math.pi * i / num_points
        z_angle = 4 * math.pi * i / num_points
        
        x = (center_x + radius * math.cos(angle)) * 1000
        y = (center_y + radius * math.sin(angle)) * 1000
        z = (base_z + z_amplitude * math.sin(z_angle)) * 1000
        
        # Move to position
        code = robot.arm.set_position(
            x=x, y=y, z=z,
            roll=180, pitch=0, yaw=0,
            speed=speed,
            wait=False  
        )
        
        if code != 0:
            print(f"WARNING: Point {i} unreachable, code={code}, pos=({x:.1f},{y:.1f},{z:.1f})")
        
        if i % 10 == 0:
            print(f"Progress: {i}/{num_points}")
        
        time.sleep(duration / num_points)
    
    print("\n✓ Circular motion complete!")
    
    # Return to safe position
    print("Returning to home...")
    robot.arm.set_position(x=300, y=0, z=200, roll=180, pitch=0, yaw=0, speed=100, wait=True)
    
except KeyboardInterrupt:
    print("\nStopped by user")
    robot.arm.set_state(4)
    
except Exception as e:
    print(f"\nError: {e}")
    robot.arm.set_state(4)

Moving robot in circular pattern with sinusoidal vertical motion...
Press Ctrl+C to stop
Current position: [227.599991, -0.0, 468.800018]
Moving to start position: (330.0, 0.0, 150.0)mm
Starting circular motion...
Progress: 0/60
Progress: 10/60
Progress: 20/60

Stopped by user
[set_state], xArm is not ready to move
